In [1]:
!pip install -U -q numpy scikit-learn pandas xgboost lightgbm category_encoders matplotlib seaborn cloudpickle shap optuna

# Enhanced ML Pipeline - Modular Feature Generation

In [2]:
import numpy as np
import pandas as pd
import os
import glob
import pickle
import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KernelDensity
import xgboost as xgb
import lightgbm as lgb
import optuna
from tabularaml.generate.features import FeatureGenerator
from tabularaml.eval.scorers import Scorer
import gc

In [3]:
import numpy as np
import pandas as pd
from typing import Generator, Tuple, Optional, Union
from sklearn.model_selection import BaseCrossValidator
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.validation import check_X_y, indexable
from sklearn.utils import check_random_state
from scipy.spatial.distance import cdist
import warnings


class SpatialTemporalKFold(BaseCrossValidator):
    """
    Spatial-Temporal Cross-Validator for geographic time-series data.
    
    This cross-validator creates folds that respect both spatial and temporal 
    dependencies in the data, preventing data leakage in spatial-temporal 
    prediction tasks like air pollution forecasting.
    
    The strategy:
    1. Creates spatial clusters using geographic coordinates (lat/lon)
    2. Creates temporal clusters using cyclical time features
    3. Combines spatial-temporal groups to ensure validation sets are 
       spatially and temporally separated from training sets
    4. Optionally stratifies by target variable ranges
    
    Parameters
    ----------
    n_splits : int, default=5
        Number of cross-validation folds
    spatial_clusters : int, default=20
        Number of spatial clusters for geographic grouping
    temporal_clusters : int, default=8
        Number of temporal clusters for time-based grouping
    lat_col : str, default='latitude'
        Column name for latitude coordinates
    lon_col : str, default='longitude' 
        Column name for longitude coordinates
    time_cols : dict, default=None
        Dictionary mapping time column names to their cycles:
        {'day_of_year': 365, 'hour': 24, 'day_of_week': 7, 'month': 12}
    stratify : bool, default=True
        Whether to stratify splits by target variable quantiles
    n_quantiles : int, default=5
        Number of quantiles for stratification (if stratify=True)
    buffer_distance : float, default=0.1
        Minimum spatial distance between train/validation clusters (degrees)
    temporal_buffer : int, default=7
        Minimum temporal distance between train/validation (in days)
    random_state : int, default=None
        Random state for reproducible splits
    shuffle : bool, default=True
        Whether to shuffle data before splitting
        
    Examples
    --------
    >>> import pandas as pd
    >>> from spatial_temporal_cv import SpatialTemporalKFold
    >>> 
    >>> # Basic usage
    >>> cv = SpatialTemporalKFold(n_splits=5, random_state=42)
    >>> X = df[['latitude', 'longitude', 'day_of_year', 'hour', 'month']]
    >>> y = df['pollution_value']
    >>> 
    >>> for train_idx, val_idx in cv.split(X, y):
    >>>     X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    >>>     y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    >>>     # Train your model...
    >>>
    >>> # Custom time columns
    >>> time_cols = {'day_of_year': 365, 'hour': 24, 'day_of_week': 7}
    >>> cv = SpatialTemporalKFold(
    >>>     n_splits=3, 
    >>>     time_cols=time_cols,
    >>>     buffer_distance=0.05,
    >>>     temporal_buffer=14
    >>> )
    """
    
    def __init__(self, 
                 n_splits: int = 5,
                 spatial_clusters: int = 20,
                 temporal_clusters: int = 8,
                 lat_col: str = 'latitude',
                 lon_col: str = 'longitude',
                 time_cols: Optional[dict] = None,
                 stratify: bool = True,
                 n_quantiles: int = 5,
                 buffer_distance: float = 0.1,
                 temporal_buffer: int = 7,
                 random_state: Optional[int] = None,
                 shuffle: bool = True):
        
        self.n_splits = n_splits
        self.spatial_clusters = spatial_clusters
        self.temporal_clusters = temporal_clusters
        self.lat_col = lat_col
        self.lon_col = lon_col
        self.time_cols = time_cols or {
            'day_of_year': 365, 'hour': 24, 'day_of_week': 7, 'month': 12
        }
        self.stratify = stratify
        self.n_quantiles = n_quantiles
        self.buffer_distance = buffer_distance
        self.temporal_buffer = temporal_buffer
        self.random_state = random_state
        self.shuffle = shuffle
        
        # Validation
        if n_splits < 2:
            raise ValueError("n_splits must be at least 2")
        if spatial_clusters < n_splits:
            warnings.warn(f"spatial_clusters ({spatial_clusters}) < n_splits ({n_splits}). "
                         "This may result in poor spatial separation.")
        if temporal_clusters < 2:
            raise ValueError("temporal_clusters must be at least 2")
    
    def get_n_splits(self, X=None, y=None, groups=None):
        """Returns the number of splitting iterations."""
        return self.n_splits
    
    def _validate_data(self, X: Union[pd.DataFrame, np.ndarray], 
                      y: Optional[Union[pd.Series, np.ndarray]] = None) -> pd.DataFrame:
        """Validate and convert input data to DataFrame."""
        if not isinstance(X, pd.DataFrame):
            if isinstance(X, np.ndarray):
                # Try to infer column names for numpy arrays
                expected_cols = [self.lat_col, self.lon_col] + list(self.time_cols.keys())
                if X.shape[1] >= len(expected_cols):
                    X = pd.DataFrame(X, columns=expected_cols[:X.shape[1]])
                else:
                    raise ValueError(f"Expected at least {len(expected_cols)} columns, got {X.shape[1]}")
            else:
                raise TypeError("X must be pandas DataFrame or numpy array")
        
        # Check required columns exist
        missing_cols = []
        if self.lat_col not in X.columns:
            missing_cols.append(self.lat_col)
        if self.lon_col not in X.columns:
            missing_cols.append(self.lon_col)
        for col in self.time_cols.keys():
            if col not in X.columns:
                missing_cols.append(col)
                
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")
            
        return X
    
    def _create_spatial_clusters(self, X: pd.DataFrame) -> np.ndarray:
        """Create spatial clusters using K-means on lat/lon coordinates."""
        coords = X[[self.lat_col, self.lon_col]]
        
        # Handle edge case where we have fewer samples than clusters
        n_clusters = min(self.spatial_clusters, len(coords))
        if n_clusters < self.spatial_clusters:
            warnings.warn(f"Reducing spatial_clusters from {self.spatial_clusters} to {n_clusters} "
                         f"due to insufficient data points.")
        
        kmeans = KMeans(
            n_clusters=n_clusters, 
            random_state=self.random_state,
            n_init=10
        )
        spatial_labels = kmeans.fit_predict(coords.copy().fillna(coords.mean()).values)
        self._spatial_centroids = kmeans.cluster_centers_
        
        return spatial_labels
    
    def _create_temporal_features(self, X: pd.DataFrame) -> np.ndarray:
        """Convert cyclical time features to circular coordinates."""
        temporal_features = []
        
        for col, cycle_length in self.time_cols.items():
            if col in X.columns:
                # Convert to radians for circular representation
                radians = 2 * np.pi * X[col] / cycle_length
                # Use sine and cosine to capture cyclical nature
                temporal_features.extend([np.sin(radians), np.cos(radians)])
        
        return np.column_stack(temporal_features) if temporal_features else np.zeros((len(X), 2))
    
    def _create_temporal_clusters(self, X: pd.DataFrame) -> np.ndarray:
        """Create temporal clusters using cyclical time features."""
        temporal_coords = self._create_temporal_features(X)
        
        # Handle edge case where we have fewer samples than clusters
        n_clusters = min(self.temporal_clusters, len(temporal_coords))
        if n_clusters < self.temporal_clusters:
            warnings.warn(f"Reducing temporal_clusters from {self.temporal_clusters} to {n_clusters} "
                         f"due to insufficient data points.")
        
        kmeans = KMeans(
            n_clusters=n_clusters,
            random_state=self.random_state,
            n_init=10
        )
        temporal_labels = kmeans.fit_predict(temporal_coords)
        self._temporal_centroids = kmeans.cluster_centers_
        
        return temporal_labels
    
    def _create_stratification_groups(self, y: np.ndarray) -> np.ndarray:
        """Create stratification groups based on target variable quantiles."""
        if not self.stratify or y is None:
            return np.zeros(len(y), dtype=int)
        
        # Create quantile-based groups
        quantiles = np.linspace(0, 1, self.n_quantiles + 1)
        quantile_values = np.quantile(y, quantiles)
        
        # Assign each sample to a quantile group
        strat_groups = np.digitize(y, quantile_values[1:-1])
        
        return strat_groups
    
    def _check_spatial_separation(self, train_spatial_groups: np.ndarray, 
                                 val_spatial_groups: np.ndarray) -> bool:
        """Check if spatial clusters are sufficiently separated."""
        if not hasattr(self, '_spatial_centroids'):
            return True
            
        train_centroids = self._spatial_centroids[train_spatial_groups]
        val_centroids = self._spatial_centroids[val_spatial_groups]
        
        # Calculate minimum distance between train and validation centroids
        distances = cdist(train_centroids, val_centroids)
        min_distance = np.min(distances)
        
        return min_distance >= self.buffer_distance
    
    def _create_combined_groups(self, X: pd.DataFrame, y: Optional[np.ndarray] = None) -> Tuple[np.ndarray, dict]:
        """Create combined spatial-temporal-stratification groups."""
        # Create individual groupings
        spatial_labels = self._create_spatial_clusters(X)
        temporal_labels = self._create_temporal_clusters(X)
        
        if y is not None:
            strat_labels = self._create_stratification_groups(y)
        else:
            strat_labels = np.zeros(len(X), dtype=int)
        
        # Combine all groupings into unique identifiers
        max_spatial = np.max(spatial_labels) + 1
        max_temporal = np.max(temporal_labels) + 1
        max_strat = np.max(strat_labels) + 1
        
        combined_groups = (spatial_labels * max_temporal * max_strat + 
                          temporal_labels * max_strat + 
                          strat_labels)
        
        # Store metadata for analysis
        metadata = {
            'spatial_labels': spatial_labels,
            'temporal_labels': temporal_labels,
            'strat_labels': strat_labels,
            'n_spatial_clusters': max_spatial,
            'n_temporal_clusters': max_temporal,
            'n_strat_groups': max_strat,
            'n_combined_groups': len(np.unique(combined_groups))
        }
        
        return combined_groups, metadata
    
    def split(self, X: Union[pd.DataFrame, np.ndarray], 
              y: Optional[Union[pd.Series, np.ndarray]] = None, 
              groups: Optional[np.ndarray] = None) -> Generator[Tuple[np.ndarray, np.ndarray], None, None]:
        """
        Generate indices to split data into training and test set.
        
        Parameters
        ----------
        X : DataFrame or array-like of shape (n_samples, n_features)
            Training data with spatial and temporal features
        y : array-like of shape (n_samples,), optional
            Target variable for stratification
        groups : array-like of shape (n_samples,), optional
            Not used, present for API compatibility
            
        Yields
        ------
        train : ndarray
            The training set indices for that split
        test : ndarray  
            The testing set indices for that split
        """
        X = self._validate_data(X, y)
        X, y = indexable(X, y)
        
        if y is not None:
            y = np.asarray(y)
            
        rng = check_random_state(self.random_state)
        n_samples = X.shape[0]
        indices = np.arange(n_samples)
        
        if self.shuffle:
            rng.shuffle(indices)
            X = X.iloc[indices].reset_index(drop=True)
            if y is not None:
                y = y[indices]
        
        # Create spatial-temporal groups
        combined_groups, metadata = self._create_combined_groups(X, y)
        unique_groups = np.unique(combined_groups)
        
        if len(unique_groups) < self.n_splits:
            raise ValueError(f"Cannot create {self.n_splits} splits with only "
                           f"{len(unique_groups)} unique spatial-temporal groups. "
                           f"Consider reducing n_splits or clustering parameters.")
        
        # Shuffle groups for random assignment to folds
        rng.shuffle(unique_groups)
        
        # Assign groups to folds using round-robin to balance sizes
        fold_groups = [[] for _ in range(self.n_splits)]
        for i, group in enumerate(unique_groups):
            fold_groups[i % self.n_splits].append(group)
        
        # Generate train/test splits
        for fold_idx in range(self.n_splits):
            test_groups = np.array(fold_groups[fold_idx])
            # Create train groups deterministically by excluding test groups
            train_groups = unique_groups[~np.isin(unique_groups, test_groups)]
            
            test_mask = np.isin(combined_groups, test_groups)
            train_mask = np.isin(combined_groups, train_groups)
            
            test_indices = indices[test_mask] if self.shuffle else np.where(test_mask)[0]
            train_indices = indices[train_mask] if self.shuffle else np.where(train_mask)[0]
            
            # Validate split quality
            if len(test_indices) == 0:
                warnings.warn(f"Fold {fold_idx} has empty test set")
                continue
            if len(train_indices) == 0:
                warnings.warn(f"Fold {fold_idx} has empty train set")
                continue
                
            yield train_indices, test_indices
    
    def get_split_info(self, X: Union[pd.DataFrame, np.ndarray], 
                      y: Optional[Union[pd.Series, np.ndarray]] = None) -> dict:
        """
        Get detailed information about the splits without generating them.
        
        Returns
        -------
        dict
            Dictionary containing split statistics and metadata
        """
        X = self._validate_data(X, y)
        if y is not None:
            y = np.asarray(y)
            
        combined_groups, metadata = self._create_combined_groups(X, y)
        
        info = {
            'n_samples': len(X),
            'n_splits': self.n_splits,
            **metadata,
            'avg_samples_per_group': len(X) / metadata['n_combined_groups'],
            'spatial_clusters_used': metadata['n_spatial_clusters'],
            'temporal_clusters_used': metadata['n_temporal_clusters']
        }
        
        # Calculate split size statistics
        unique_groups = np.unique(combined_groups)
        fold_groups = [[] for _ in range(self.n_splits)]
        for i, group in enumerate(unique_groups):
            fold_groups[i % self.n_splits].append(group)
            
        fold_sizes = []
        for fold_idx in range(self.n_splits):
            test_groups = np.array(fold_groups[fold_idx])
            test_mask = np.isin(combined_groups, test_groups)
            fold_sizes.append(np.sum(test_mask))
            
        info.update({
            'fold_sizes': fold_sizes,
            'min_fold_size': min(fold_sizes),
            'max_fold_size': max(fold_sizes),
            'fold_size_std': np.std(fold_sizes)
        })
        
        return info


class StratifiedSpatialTemporalKFold(SpatialTemporalKFold):
    """
    Stratified version that ensures better target distribution balance.
    
    This extends SpatialTemporalKFold with enhanced stratification that
    tries to maintain similar target distributions across folds while
    still respecting spatial-temporal constraints.
    """
    
    def __init__(self, **kwargs):
        # Force stratification on
        kwargs['stratify'] = True
        super().__init__(**kwargs)
    
    def split(self, X, y=None, groups=None):
        """Split with enhanced stratification checking."""
        if y is None:
            raise ValueError("StratifiedSpatialTemporalKFold requires y for stratification")
            
        # Generate base splits
        for train_idx, test_idx in super().split(X, y, groups):
            # Check stratification quality
            y_train, y_test = y[train_idx], y[test_idx]
            
            # Calculate quantile distributions
            train_quantiles = np.quantile(y_train, [0.25, 0.5, 0.75])
            test_quantiles = np.quantile(y_test, [0.25, 0.5, 0.75]) 
            
            # Check if distributions are too different
            max_diff = np.max(np.abs(train_quantiles - test_quantiles))
            if max_diff > 2 * np.std(y):
                warnings.warn(f"Large distribution difference detected: {max_diff:.4f}")
                
            yield train_idx, test_idx

In [4]:
from sklearn.model_selection import BaseCrossValidator
import numpy as np

class FixedWindowTimeSeriesSplit(BaseCrossValidator):
    """
    Custom time-series cross-validator with fixed-size test windows.
    Ensures every fold has meaningful training data and proper temporal ordering.
    
    Parameters
    ----------
    n_splits : int
        Number of folds. Must be at least 1.
    test_size : int
        Number of samples in each test fold.
    gap : int, default=0
        Number of samples to exclude between train and test sets.
    min_train_size : int, default=None
        Minimum number of training samples required. If None, defaults to test_size.
    """
    
    def __init__(self, n_splits=5, test_size=2700, gap=0, min_train_size=None):
        if n_splits < 1:
            raise ValueError("n_splits must be at least 1.")
        if test_size < 1:
            raise ValueError("test_size must be at least 1.")
        if gap < 0:
            raise ValueError("gap must be non-negative.")
        
        self.n_splits = n_splits
        self.test_size = test_size
        self.gap = gap
        self.min_train_size = min_train_size or test_size
        
        if self.min_train_size < 1:
            raise ValueError("min_train_size must be at least 1.")
    
    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        
        # Check if we have enough data for at least one split
        min_required = self.min_train_size + self.gap + self.test_size
        if min_required > n_samples:
            raise ValueError(
                f"Not enough samples. Need at least {min_required} samples "
                f"(min_train_size={self.min_train_size} + gap={self.gap} + test_size={self.test_size}), "
                f"but got {n_samples}."
            )
        
        indices = np.arange(n_samples)
        
        if self.n_splits == 1:
            # Single window: place test at the end, ensure minimum training size
            test_end = n_samples
            test_start = test_end - self.test_size
            train_end = test_start - self.gap
            
            # Ensure we have minimum training size
            if train_end < self.min_train_size:
                train_end = self.min_train_size
                test_start = train_end + self.gap
                test_end = test_start + self.test_size
                
                # Check if this fits within our data
                if test_end > n_samples:
                    raise ValueError(
                        f"Cannot fit single split with constraints. "
                        f"Need {self.min_train_size + self.gap + self.test_size} samples, got {n_samples}."
                    )
            
            train_idx = indices[:train_end]
            test_idx = indices[test_start:test_end]
            yield train_idx, test_idx
            return
        
        # For multiple splits, distribute test windows
        # Last test window ends at n_samples, work backwards
        test_windows = []
        
        # Calculate positions for test windows
        # We want to distribute them evenly in the available space
        latest_test_end = n_samples
        earliest_test_start = self.min_train_size + self.gap
        
        # Available space for test window starts
        available_space = latest_test_end - self.test_size - earliest_test_start
        
        if available_space < 0:
            raise ValueError(
                "Cannot create requested splits. Try reducing n_splits, test_size, or min_train_size."
            )
        
        # Calculate step size between test windows
        if self.n_splits == 1:
            step = 0
        else:
            step = available_space / (self.n_splits - 1)
        
        # Generate test windows from last to first
        for i in range(self.n_splits):
            # Calculate test window position
            test_start = int(earliest_test_start + i * step)
            test_end = test_start + self.test_size
            
            # Ensure test window doesn't exceed data bounds
            if test_end > n_samples:
                test_end = n_samples
                test_start = test_end - self.test_size
            
            # Calculate training end (before gap)
            train_end = test_start - self.gap
            
            # Ensure minimum training size
            if train_end < self.min_train_size:
                raise ValueError(
                    f"Split {i+1} would have insufficient training data. "
                    f"Try reducing n_splits or min_train_size."
                )
            
            train_idx = indices[:train_end]
            test_idx = indices[test_start:test_end]
            
            yield train_idx, test_idx

In [5]:
def rmse_exp(y_true, y_pred):
    return np.exp(-np.sqrt(mean_squared_error(y_true, y_pred))/100)

def competition_score(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return np.exp(-rmse / 100)

def create_cyclical_features(df):
    df_copy = df.copy()
    
    if 'hour' in df_copy.columns:
        df_copy['hour_sin'] = np.sin(2 * np.pi * df_copy['hour'] / 24.0)
        df_copy['hour_cos'] = np.cos(2 * np.pi * df_copy['hour'] / 24.0)
    
    if 'day_of_week' in df_copy.columns:
        df_copy['dow_sin'] = np.sin(2 * np.pi * df_copy['day_of_week'] / 7.0)
        df_copy['dow_cos'] = np.cos(2 * np.pi * df_copy['day_of_week'] / 7.0)
    
    if 'day_of_year' in df_copy.columns:
        df_copy['doy_sin'] = np.sin(2 * np.pi * df_copy['day_of_year'] / 365.0)
        df_copy['doy_cos'] = np.cos(2 * np.pi * df_copy['day_of_year'] / 365.0)
    
    columns_to_drop = ['hour', 'day_of_week', 'day_of_year', 'month']
    existing_columns_to_drop = [col for col in columns_to_drop if col in df_copy.columns]
    if existing_columns_to_drop:
        df_copy = df_copy.drop(columns=existing_columns_to_drop)
    
    return df_copy

In [6]:
def run_single_feature_generation(X_train, y_train, save_dir="./features", run_id=1, max_new_feats=2000, n_generations=2000):
    os.makedirs(save_dir, exist_ok=True)
    
    splitter = SpatialTemporalKFold(n_splits=5, random_state=42)
    rmse_exp_scorer = Scorer(name="rmse_exp", scorer=rmse_exp, greater_is_better=True, extra_params={}, from_probs=False)
    
    print(f"Running feature generation run {run_id}...")
    
    generator = FeatureGenerator(
        task="regression",
        scorer=rmse_exp_scorer,
        max_new_feats=max_new_feats,
        cv=splitter,
        n_generations=n_generations,
        save_path=f"{save_dir}/feature_generator_run_{run_id}.pkl",
    )
    
    results = generator.search(X_train, y_train)
    X_generated = generator.transform(X_train)
    
    print(f"Run {run_id} complete: {X_generated.shape[1]} features generated")
    return X_generated, generator

In [7]:
def load_all_feature_generators(feature_dir):
    generator_files = glob.glob(os.path.join(feature_dir, "*.pkl"))
    if not generator_files:
        raise ValueError(f"No feature generator files found in {feature_dir}")
    
    generators = []
    print(f"Loading {len(generator_files)} feature generators from {feature_dir}: {sorted(generator_files)}.")
    for file_path in sorted(generator_files):
        generator = FeatureGenerator.load(file_path)
        generators.append(generator)

    return generators

def combine_feature_generators(generators, X_train, X_test=None):
    print(f"Combining features from {len(generators)} generators...")
    X_train = X_train.copy()
    if X_test is not None:
        X_test = X_test.copy()
    
    for i, generator in enumerate(generators):
        X_train = generator.fit_transform(X_train)
        if X_test is not None:
            X_test = generator.transform(X_test)
    
    if X_test is not None:
        return X_train, X_test
    return X_train

In [8]:
def enhanced_objective(trial, X_base, y_train, tss, n_clusters_range=(20, 50)):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 30),
        'gamma': trial.suggest_float('gamma', 0, 20),
        'subsample': trial.suggest_float('subsample', 0.3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.3, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.3, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 100.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 100.0, log=True),
        'max_leaves': trial.suggest_int('max_leaves', 0, 2000),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'max_bin': trial.suggest_int('max_bin', 32, 512),
        'objective': 'reg:squarederror',
        'enable_categorical': True,
        'random_state': 42,
        'n_jobs': -1
    }
    
    n_clusters = trial.suggest_int('n_clusters', n_clusters_range[0], n_clusters_range[1])
    
    X_base_cyclic = create_cyclical_features(X_base.copy())
    y = np.log1p(y_train.copy())
    
    fold_scores = []
    
    for n_fold, (train_idx, valid_idx) in enumerate(tss.split(X_base_cyclic, y)):
        X_train_fold = X_base_cyclic.iloc[train_idx].copy()
        X_valid_fold = X_base_cyclic.iloc[valid_idx].copy()
        y_train_fold = y.iloc[train_idx]
        y_valid_fold = y.iloc[valid_idx]
        
        if 'latitude' in X_train_fold.columns and 'longitude' in X_train_fold.columns:
            lat_mean = X_train_fold['latitude'].mean()
            lon_mean = X_train_fold['longitude'].mean()
            
            train_coords_temp = X_train_fold[['latitude', 'longitude']].copy()
            valid_coords_temp = X_valid_fold[['latitude', 'longitude']].copy()
            
            train_coords_temp['latitude'].fillna(lat_mean, inplace=True)
            train_coords_temp['longitude'].fillna(lon_mean, inplace=True)
            valid_coords_temp['latitude'].fillna(lat_mean, inplace=True)
            valid_coords_temp['longitude'].fillna(lon_mean, inplace=True)
            
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
            kmeans.fit(train_coords_temp)
            
            X_train_fold['cluster'] = kmeans.predict(train_coords_temp)
            X_valid_fold['cluster'] = kmeans.predict(valid_coords_temp)
        
        model = xgb.XGBRegressor(**params)
        model.fit(X_train_fold, y_train_fold, verbose=False)
        
        y_pred_fold = model.predict(X_valid_fold)
        y_pred_orig_scale = np.expm1(y_pred_fold)
        y_valid_orig_scale = np.expm1(y_valid_fold)
        
        exp_score = competition_score(y_valid_orig_scale, y_pred_orig_scale)
        fold_scores.append(exp_score)
    
    return np.mean(fold_scores)

In [9]:
def train_final_model(X_train, y_train, params, model_type='xgb'):
    X_train_cyclic = create_cyclical_features(X_train.copy())
    
    n_clusters = params.get('n_clusters', 30)
    if 'latitude' in X_train_cyclic.columns and 'longitude' in X_train_cyclic.columns:
        lat_mean = X_train_cyclic['latitude'].mean()
        lon_mean = X_train_cyclic['longitude'].mean()
        
        coords_temp = X_train_cyclic[['latitude', 'longitude']].copy()
        coords_temp['latitude'].fillna(lat_mean, inplace=True)
        coords_temp['longitude'].fillna(lon_mean, inplace=True)
        
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
        kmeans.fit(coords_temp)
        X_train_cyclic['cluster'] = kmeans.predict(coords_temp)
    
    y_train_log = np.log1p(y_train)
    
    if model_type == 'xgb':
        model_params = {k: v for k, v in params.items() if k != 'n_clusters'}
        model = xgb.XGBRegressor(**model_params)
        model.fit(X_train_cyclic, y_train_log)
    elif model_type == 'lgb':
        lgb_params = {
            'n_estimators': params.get('n_estimators', 1000),
            'learning_rate': params.get('learning_rate', 0.05),
            'num_leaves': 2 ** params.get('max_depth', 6) - 1,
            'feature_fraction': params.get('colsample_bytree', 0.8),
            'bagging_fraction': params.get('subsample', 0.8),
            'bagging_freq': 1,
            'lambda_l1': params.get('reg_alpha', 0),
            'lambda_l2': params.get('reg_lambda', 1),
            'min_data_in_leaf': int(params.get('min_child_weight', 1)),
            'random_state': 42,
            'verbose': -1
        }
        model = lgb.LGBMRegressor(**lgb_params)
        model.fit(X_train_cyclic, y_train_log, verbose=-1)
    
    model.kmeans_ = kmeans if 'kmeans' in locals() else None
    model.lat_mean_ = lat_mean if 'lat_mean' in locals() else None
    model.lon_mean_ = lon_mean if 'lon_mean' in locals() else None
    
    return model

In [ ]:
def create_model_ensemble(X_train, y_train, X_test, best_params, tss):
    predictions = []
    
    # Apply cyclical features and clustering to test data
    X_test_processed = create_cyclical_features(X_test.copy())
    
    # Add clustering to test data using the same parameters as training
    n_clusters = best_params.get('n_clusters', 30)
    if 'latitude' in X_test_processed.columns and 'longitude' in X_test_processed.columns:
        # We need to get the lat/lon means from training data for consistency
        X_train_temp = create_cyclical_features(X_train.copy())
        lat_mean = X_train_temp['latitude'].mean()
        lon_mean = X_train_temp['longitude'].mean()
        
        # Prepare test coordinates
        test_coords_temp = X_test_processed[['latitude', 'longitude']].copy()
        test_coords_temp['latitude'].fillna(lat_mean, inplace=True)
        test_coords_temp['longitude'].fillna(lon_mean, inplace=True)
        
        # Prepare training coordinates for fitting KMeans
        train_coords_temp = X_train_temp[['latitude', 'longitude']].copy()
        train_coords_temp['latitude'].fillna(lat_mean, inplace=True)
        train_coords_temp['longitude'].fillna(lon_mean, inplace=True)
        
        # Fit KMeans on training data and predict on test data
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
        kmeans.fit(train_coords_temp)
        X_test_processed['cluster'] = kmeans.predict(test_coords_temp)
    
    print("Training XGBoost with best parameters...")
    xgb_model = train_final_model(X_train, y_train, best_params, model_type='xgb')
    pred_xgb = xgb_model.predict(X_test_processed)
    predictions.append(('xgb_best', pred_xgb, 0.7))
    
    print("Training LightGBM...")
    lgb_model = train_final_model(X_train, y_train, best_params, model_type='lgb')
    pred_lgb = lgb_model.predict(X_test_processed)
    predictions.append(('lgb', pred_lgb, 0.2))
    
    print("Training XGBoost variant...")
    xgb_variant_params = best_params.copy()
    xgb_variant_params['max_depth'] = min(best_params['max_depth'] + 2, 15)
    xgb_variant_params['learning_rate'] = best_params['learning_rate'] * 0.8
    xgb_variant_model = train_final_model(X_train, y_train, xgb_variant_params, model_type='xgb')
    pred_xgb_variant = xgb_variant_model.predict(X_test_processed)
    predictions.append(('xgb_variant', pred_xgb_variant, 0.1))
    
    final_pred = np.zeros(len(X_test_processed))
    for name, pred, weight in predictions:
        final_pred += weight * pred
    
    return final_pred, predictions

In [11]:
def advanced_post_processing(train_df, y_train, test_df, predictions):
    train_quantiles = np.percentile(y_train, [10, 25, 50, 75, 90])
    pred_quantiles = np.percentile(predictions, [10, 25, 50, 75, 90])
    
    if abs(train_quantiles[2] - pred_quantiles[2]) > 0.1 * train_quantiles[2]:
        scale = train_quantiles[2] / pred_quantiles[2]
        predictions_adjusted = predictions * scale
    else:
        predictions_adjusted = predictions
    
    test_lat_min, test_lat_max = test_df['latitude'].min(), test_df['latitude'].max()
    train_lat_min, train_lat_max = train_df['latitude'].min(), train_df['latitude'].max()
    
    if test_lat_min < train_lat_min or test_lat_max > train_lat_max:
        extreme_mask = ((test_df['latitude'] < train_lat_min) | (test_df['latitude'] > train_lat_max))
        if extreme_mask.any():
            mean_pred = predictions_adjusted[~extreme_mask].mean()
            predictions_adjusted[extreme_mask] = (0.7 * predictions_adjusted[extreme_mask] + 0.3 * mean_pred)
    
    return predictions_adjusted

In [12]:
def enhanced_pipeline(train_df, test_df, target_col='pollution_value', n_trials=1000, feature_dir=None):
    print("="*60)
    print("ENHANCED PIPELINE - BUILDING ON SUCCESS")
    print("="*60)
    
    X_train = train_df.drop(target_col, axis=1)
    y_train = train_df[target_col]
    X_test = test_df.copy()
    
    test_ids = test_df['id'].values
    
    if feature_dir and os.path.exists(feature_dir):
        print(f"\nLoading existing feature generators from {feature_dir}...")
        generators = load_all_feature_generators(feature_dir)
        X_train_generated, X_test_generated = combine_feature_generators(generators, X_train, X_test)
    else:
        print("\nNo existing feature generators found. Please run Stage 1 first.")
        return None, None, None
    
    print(f"Total features after generation: {X_train_generated.shape[1]}")
    
    print("\nOptimizing hyperparameters...")
    
    tss = FixedWindowTimeSeriesSplit(n_splits=5, test_size=2700, gap=0, min_train_size=2700)
    
    sampler = optuna.samplers.TPESampler(multivariate=True, group=True, n_startup_trials=20, constant_liar=True, seed=42)
    study = optuna.create_study(direction="maximize", study_name="xgboost_optimization_enhanced", sampler=sampler, storage="sqlite:///xgb_optuna_enhanced.db", load_if_exists=True)
    
    objective_func = lambda trial: enhanced_objective(trial, X_train_generated, y_train, tss, n_clusters_range=(20, 50))
    study.optimize(objective_func, n_trials=n_trials)
    
    best_params = study.best_params.copy()
    print(f"\nBest score: {study.best_value:.4f}")
    
    print("\nGenerating final predictions...")
    final_predictions, model_predictions = create_model_ensemble(X_train_generated, y_train, X_test_generated, best_params, tss)
    
    print("\nApplying post-processing...")
    final_predictions = np.maximum(final_predictions, 0)
    final_predictions = advanced_post_processing(train_df, y_train, test_df, final_predictions)
    
    submission = pd.DataFrame({'id': test_ids, 'pollution_value': final_predictions})
    
    return submission, study, best_params

In [13]:
# Load data
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
year = 2023
train_df['datetime'] = pd.to_datetime(train_df['day_of_year'], format='%j', errors='coerce') \
                       + pd.to_timedelta(train_df['hour'], unit='h')
train_df['datetime'] = train_df['datetime'].apply(
    lambda dt: dt.replace(year=year) if pd.notnull(dt) else dt
)
train_df = train_df.sort_values(by='datetime')
train_df = train_df.drop(columns='datetime')
train_df.reset_index(drop=True, inplace=True) # CRUCIAL #

X_train = train_df.drop('pollution_value', axis=1)
y_train = train_df['pollution_value']

In [14]:
# # Stage 1: Run feature generation multiple times
# for run_id in range(1, 4):
#     X_generated, generator = run_single_feature_generation(
#         X_train, y_train, 
#         save_dir="model_rredone", 
#         run_id=run_id,
#         max_new_feats=2000,
#         n_generations=2000
#     )

In [ ]:
# Stage 2: Run complete pipeline
submission, study, best_params = enhanced_pipeline(
    train_df, 
    test_df,
    target_col='pollution_value',
    n_trials=10,
    feature_dir="model_rredone"
)

submission.to_csv('submission_enhanced.csv', index=False)
print("Submission saved!")

ENHANCED PIPELINE - BUILDING ON SUCCESS

Loading existing feature generators from model_rredone...
Loading 1 feature generators from model_rredone: ['model_rredone/feature_generator_1.pkl'].
Combining features from 1 generators...
Total features after generation: 17

Optimizing hyperparameters...


[I 2025-07-16 03:21:34,336] Using an existing study with name 'xgboost_optimization_enhanced' instead of creating a new one.
